## 4.5 PUMApp

W tej aplikacji wykorzystamy `RecyclerView`, `Drawer`, `Jetpack Navigation` oraz `Materials` do stworzenia prostej aplikacji będącej mocno uproszczoną wersją tego repozytorium.

### **Fragmenty**

Rozpocznijmy od dodania fragmentów (**Blank Fragment**) z których będzie składała się nasza aplikacja.

- `StartFragment` - fragment domowy
- `ModuleListFragment` - fragment zawierający listę modułów
- `ModuleFragment` - fragment zawierający zawartość modułu
- `LectureFragment` - fragment zawierający informacje o wykładzie
- `LabListFragment` - fragment zawierający listę wszystkich labów przeznaczonych do tego laboratorium
- `AppListFragment` - fragment zawierający listę wszystkich aplikacji przeznaczonych do tego laboratorium
- `AboutFragment` - fragment zawierający podstawowe informacje o aplikacji
- `SettingsFragment` - fragment zawierający ustawienia aplikacji - w tej aplikacji dodamy możliwość przełączenia na ciemny motyw

### **Nawigacja**

Dodajmy do aplikacji nawigację. Do skryptów `Gradle(Project)` dodajemy

In [ ]:
buildscript {
    repositories {
        google()
    }
    dependencies {
        def nav_version = "2.5.0"
        classpath "androidx.navigation:navigation-safe-args-gradle-plugin:$nav_version"
    }
}

Do `Gradle(Module)`

In [ ]:
plugins {
    id 'com.android.application'
    id 'org.jetbrains.kotlin.android'
    id 'androidx.navigation.safeargs.kotlin'
}

dependencies {

    def nav_version = "2.5.0"
    implementation "androidx.navigation:navigation-fragment:$nav_version"
    implementation "androidx.navigation:navigation-ui:$nav_version"
}

Następnie synchronizujemy projekt i przechodzimy do utworzenia grafu.

<img src="https://files.fm/thumb_show.php?i=375bnxj9c" width="400" />

Zauważmy że `AboutFragment` oraz `SettingsFragment` nie posiadają żadnej zdefiniowanej akcji - nawigację do tych fragmentów dodamy za pomocą `Drawer`.

Dodajmy `Menu` dla `Drawer`. Otwieramy menu kontekstowe na katalogu `res`, następnie **New -> Android Resource File**, jako **Resource Type** wybieram `Menu` - plik nazwijmy `drawer_menu`. Dodamy w nim dwa wpisy `<item>`

```xml
<?xml version="1.0" encoding="utf-8"?>
<menu xmlns:android="http://schemas.android.com/apk/res/android">
    <item
        android:id="@+id/aboutFragment"
        android:title="About" />

    <item
        android:id="@+id/settingsFragment"
        android:title="Settings" />
</menu>
```

Istotną rzeczą są `id` - muszą być takie same jak w `navigation.xml` pola `id` nadane odpowiednim fragmentom. Tutaj chcemy zapewnić możliwość nawigacji z menu bocznego do dwóch fragmentów, więc dodajemy dwa pola o odpowiednich `id`.

Kolejnym krokiem będzie modyfikacja layoutu `MainActivity`. Naszym głównym elementem będzie `DrawerLayout` - ponieważ chcemy móc otworzyć panel boczny, tutaj będzie on powiązany z aktywnością główną aplikacji.

```xml
<androidx.drawerlayout.widget.DrawerLayout xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:tools="http://schemas.android.com/tools"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    android:layout_width="match_parent"
    android:layout_height="match_parent"
    android:id="@+id/drawerLayoutID"
    tools:context=".MainActivity">
```

Następnie potrzebujemy `FragmentContainer` - jak w poprzednich aplikacjach

```xml
    <LinearLayout
        android:layout_width="match_parent"
        android:layout_height="match_parent"
        android:orientation="vertical">

        <androidx.fragment.app.FragmentContainerView
            android:id="@+id/nav_host_fragment"
            android:name="androidx.navigation.fragment.NavHostFragment"
            android:layout_width="match_parent"
            android:layout_height="match_parent"
            app:defaultNavHost="true"
            app:navGraph="@navigation/navigation" />
    </LinearLayout>
```

Ostatnim elementem jest `NavigationView`, który pozwoli dodać wcześniej utworzony `drawer_menu` i wyświetlić panel boczny

```xml
    <com.google.android.material.navigation.NavigationView
        android:id="@+id/navigation_drawerID"
        android:layout_width="wrap_content"
        android:layout_height="match_parent"
        android:layout_gravity="start"
        app:menu="@menu/drawer_menu"/>

</androidx.drawerlayout.widget.DrawerLayout>
```



Ostatnim krokiem w tworzeniu nawigacji w tym projekcie będzie spięcie wszystkiego w `MainActivity`. W pierwszym kroku musimy posiadać instancję `NavController`

In [ ]:
private NavController navController;

@Override
protected void onCreate(Bundle savedInstanceState) {
    super.onCreate(savedInstanceState);
    setContentView(R.layout.activity_main);

    NavHostFragment navHostFragment = (NavHostFragment) getSupportFragmentManager()
            .findFragmentById(R.id.nav_host_fragment);

    if (navHostFragment != null) {
        navController = NavHostFragment.findNavController(navHostFragment);
    }
}

Następnie definiujemy `AppBarConfiguration` w którym łączymy `NavController` z `DrawerLayout`

In [ ]:
private NavController navController;
private AppBarConfiguration appBarConfiguration;

@Override
protected void onCreate(Bundle savedInstanceState) {
    super.onCreate(savedInstanceState);
    setContentView(R.layout.activity_main);

    NavHostFragment navHostFragment = (NavHostFragment) getSupportFragmentManager()
            .findFragmentById(R.id.nav_host_fragment);

    if (navHostFragment != null) {
        navController = NavHostFragment.findNavController(navHostFragment);
    }

    DrawerLayout drawerLayout = findViewById(R.id.drawerLayoutID);

    appBarConfiguration =  new AppBarConfiguration.Builder(navController.getGraph())
            .setOpenableLayout(drawerLayout)
            .build();
}

W metodzie `onCreate` łączymy domyślny `ActionBar` z `NavController`

In [ ]:
NavigationUI.setupActionBarWithNavController(this, navController, appBarConfiguration);

oraz `NavigationView` z `NavController`

In [ ]:
NavigationView navigationView = findViewById(R.id.navigation_drawerID);
NavigationUI.setupWithNavController(navigationView, navController);

Dodamy implementacje dwóch metod. Pierwszą jest `onSupportNavigateUp` - pozwala ona na zmianę ikony szuflady na ikonę powrotu po opuszczeniu głównego ekranu aplikacji - szufladę zawsze można otworzyć przez przeciągnięcie.

In [ ]:
@Override
public boolean onSupportNavigateUp() {
    return NavigationUI.navigateUp(navController, appBarConfiguration) || super.onSupportNavigateUp();
}

Następną metodą jest `onOptionsItemSelected` - pozwala ona na nawigację bez definiowania akcji pod warunkiem zgodności `id` w plikach `drawer_menu` oraz `navigation`

In [ ]:
@Override
public boolean onOptionsItemSelected(@NonNull MenuItem item) {
    return NavigationUI.onNavDestinationSelected(item, navController) || super.onOptionsItemSelected(item);
}

Możemy przetestować działanie `Drawer`

<img src="https://media1.giphy.com/media/v98tLyiUpSyfTG9VmD/giphy.gif?cid=790b76110cf80882ea0dfcb8fdb4c8c397f8a3d45e0401df&rid=giphy.gif&ct=g" width="150" />